In [ ]:
import os
import pandas as pd
import tensorflow as tf
import time
import datetime

# путь к отфильтрованному!!! датасету
dataset_path = 'B:\\DATASET\\iot_intrusion_dataset\\filtered\\'

# путь к исполняемому файлу tshark
tshark_path = 'A:\\Wireshark\\tshark'

# имена файлов, содержащих обычный трафик
dataset_files_norm = [
    'benign-dec'
]

# имена файлов, содержащих трафик с атаками
# (рассматривались только атаки synflooding и ackflooding)
dataset_files_attack = [
    'dos-synflooding-1-dec',
    'dos-synflooding-2-dec',
    'dos-synflooding-3-dec',
    'dos-synflooding-4-dec',
    'dos-synflooding-5-dec',
    'dos-synflooding-6-dec',
    'mirai-ackflooding-1-dec',
    'mirai-ackflooding-2-dec',
    'mirai-ackflooding-3-dec',
    'mirai-ackflooding-4-dec'
]

dataset_files_norm = [dataset_path + file_name for file_name in dataset_files_norm]
dataset_files_attack = [dataset_path + file_name for file_name in dataset_files_attack]

In [ ]:
# pcap -> csv
# если нужные csv-файлы уже сгенерировыны, пропустить!

# поля, которые будут использоваться для обучения и дальнейшего выявления
# атак, нужно подбирать самостоятельно: мой вариант скорее демонстрационный, чем
# практически эффективный

# список полей (формат полей из Wireshark)
fields = ['tcp.srcport', 'tcp.dstport', 'ip.len', 'ip.id']

fields_str = ''
for field in fields:
    fields_str += '-e ' + field + ' '

for file_path in dataset_files_norm + dataset_files_attack:
    os.system(tshark_path + ' -r {}.pcap -T fields {}-E header=y -E separator=, -E occurrence=f > {}.csv'.format(file_path, fields_str, file_path))

for file_path in dataset_files_norm:
    df = pd.read_csv(file_path + '.csv')
    df['attack'] = '0'
    df.to_csv(file_path + '.csv', index=False)

for file_path in dataset_files_attack:
    df = pd.read_csv(file_path + '.csv')
    df['attack'] = '1'
    df.to_csv(file_path + '.csv', index=False)

In [ ]:
# получить список всех файлов

dataset_files_all = [file_path + '.csv' for file_path in dataset_files_norm + dataset_files_attack]
print('Список всех файлов датасета:\n', dataset_files_all)

In [ ]:
# получить общий датафрейм

summary_dataframe = pd.DataFrame()
for file_path in dataset_files_all:
    temp_dataframe = pd.read_csv(file_path)
    summary_dataframe = summary_dataframe.append(temp_dataframe, ignore_index=True)

summary_dataframe['ip.id'] = summary_dataframe['ip.id'].apply(int, base=16)
print('Общий датафрейм:\n', summary_dataframe)

In [ ]:
# получить тренировочный датафрейм

train_dataframe = summary_dataframe.sample(int(0.7 * summary_dataframe.shape[0]))

print('Тренировочный датафрейм:\n', train_dataframe)
print('Размер:\n', train_dataframe.shape)

In [ ]:
# составить тренировочный датасет

train_target = train_dataframe.pop('attack')
train_dataset = tf.data.Dataset.from_tensor_slices((train_dataframe.values, train_target.values))
print('Пример тренировочного датасета:')
for feat, targ in train_dataset.take(5):
    print ('Значение: {}, цель: {}'.format(feat, targ))

In [ ]:
train_dataset = train_dataset.shuffle(len(train_dataframe)).batch(1)
test_dataset = test_dataset.shuffle(len(test_dataframe))

def get_compiled_model():
    model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(160, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

print('---ОБУЧЕНИЕ---')
timer = time.process_time()
model = get_compiled_model()
model.fit(train_dataset, epochs=3)
timer = time.process_time() - timer
print('Время обучения =', timer, ' секунд')

In [ ]:
print('---ТЕСТИРОВАНИЕ---')
timer = time.process_time()
test_loss, test_accuracy = model.evaluate(test_dataframe, test_target, verbose=1)
timer = time.process_time() - timer
print('Потери = {}, точность = {}'.format(test_loss, test_accuracy))
print('Время тестирования =', timer, ' секунд')